## 데이터 로더(Data Loaders)

이 노트북은 `pyTigerGraph` 에서 **edge loader** 의 사용을 보여줍니다. 데이터 로더의 역할은 `TigerGraph` 데이터베이스에서 데이터를 가져오는 것입니다. 현재 다음 데이터 로더가 제공됩니다.

* EdgeLoader, 간선의 배치(batch)를 반환합니다.
* VertexLoader, 정점의 배치(batch)를 반환합니다.
* GraphLoader, 무작위로 샘플링된(연결이 끊긴) 하위 그래프(subgraphs)를 pandas의 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.
* NeighborLoader, 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL`  형식으로 반환 합니다.
* EdgeNeighborLoader, 간선들에서 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.

위의 모든 데이터 로더는 모든 배치를 HTTP 응답(기본값)으로 가져오거나 Kafka를 통해 모든 배치를 스트리밍할 수 있습니다. 전자의 메커니즘은 작은 그래프로 테스트하기에 좋고 빠르지만 데이터 크기 제한이 2GB입니다. 큰 그래프의 경우 크기 제한 및 네트워크 연결 문제로 인해 HTTP 채널이 실패할 수 있습니다. Kafka를 통한 스트리밍은 데이터 견고성과 확장성을 위해 제공됩니다. 또한 Kafka는 다중 소비자(multi-consumer) 사용 사례에 탁월하며 동일한 데이터의 다중 소비자가 있는 경우 모델 검색 또는 하이퍼파라미터 튜닝에 효율적입니다.

데이터 로더는 동종 및 이종 그래프를 모두 지원합니다. 기본적으로 모든 정점 및 간선 유형에서 로드하고 그래프를 동종 그래프로 처리합니다. 그러나 사용자가 로드할 정점 및 간선 유형과 각 유형에서 로드할 속성을 지정할 수도 있습니다. 이 방법으로 사용자는 이기종 그래프 출력을 얻을 수 있습니다.

**NOTE**: 현재 ML Workbench에서 제공하는 모든 기능을 사용하려면 데이터베이스를 한 번만 활성화(activage)해야 합니다. ML Workbench on Cloud를 사용하는 경우 activator가 포함되며 아래 셀을 실행(먼저 주석 해제)하여 활성화(activage)할 수 있습니다. 다른 버전의 Workbench의 경우 https://act.tigergraphlabs.com 에서 활성기(activator)를 다운로드할 수 있습니다. 자세한 지침은 해당 웹사이트에도 포함되어 있습니다.


In [ ]:
# 아래 주석을 제거하고 필요한 정보를 입력하십시오. 자세한 지침은 https://act.tigergraphlabs.com을 참조하십시오.
# !mlwb activate [데이터베이스 주소] -u [사용자 이름] -p [비밀번호] -s [비밀]

### 데이터베이스 연결 

이 `TigerGraphConnection` 클래스는 TigerGraph 데이터베이스에 대한 연결을 나타냅니다. 후드 아래에는 데이터베이스와 통신하는 데 필요한 정보가 저장됩니다. 꽤 많은 데이터베이스 작업을 수행할 수 있습니다. 자세한 내용은 해당 [문서](https://docs.tigergraph.com/pytigergraph/current/intro/) 를 참조하십시오.

**Note**: 2022년 7월 5일 이후에 생성된 TG 클라우드 DB는 사용자 이름/비밀번호 대신 비밀 번호가 필요합니다. 그렇지 않으면 비워 둘 수 있습니다.


In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.  
)

<span style="color:red">아래 셀의 주석을 제거하고 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하기 위해 실행합니다.</span>. 
* 이것은 tgcloud의 모든 데이터베이스에 필요합니다.
* `<secret>`은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets 를 참조하세요.
* secret을 모르는 경우  `secret=conn.createSecret()`를 사용하여 secret을 만들 수 있습니다.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# 모든 정점 유형에 대한 정점 수
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# 모든 간선 유형에 대한 간선 수
conn.getEdgeCount()

{'Cite': 10556}

### 에지 로더 (Edge Loader)

`EdgeLoader` pulls batches of edges from database. Specifically, it divides edges into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which edges are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph,
the initialization might take a minute as it installs the corresponding
query to the database and optimizes it. However, the query installation only
needs to be done once, so it will take no time when you initialize the loader
on the same TG graph again.

There are two ways to use the data loader. See
[here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all edges at once (`num_batches=1`), there will be only one batch (of all the edges) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list or dict, optional):
        Edge attributes to be included. If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

`EdgeLoader`는 데이터베이스에서 가장자리 배치를 가져옵니다. 특히 간선을 `num_batches`로 분할 하고 각 배치를 개별적으로 반환합니다. `filter_by`에 제공된 부울 속성은 포함된 가선을 나타냅니다. 임의 배치가 필요한 경우 `shuffle`을 True로 설정하십시오.

**Note**: TigerGraph의 그래프에서 로더를 처음 초기화하는 경우 해당 쿼리를 데이터베이스에 설치하고 최적화하므로 초기화에 1분이 걸릴 수 있습니다. 하지만 쿼리 설치는 한 번만 하면 되므로 동일한 TG 그래프에서 다시 로더를 초기화할 때 시간이 걸리지 않습니다.

데이터 로더를 사용하는 방법에는 두 가지가 있습니다. 예를 보려면 [여기](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) 를 참조 하십시오.

* 첫째, iterable로 사용할 수 있습니다. 즉, 루프를 통해 모든 데이터 배치를 가져올 수 있습니다. 모든 에지를 한 번에 로드하면(`num_batches=1`), 반복자에는 (모든 에지 중) 배치가 하나만 있습니다.
* 둘째, 클래스의 `data`속성에 직접 액세스할 수 있습니다. 로드할 데이터 배치가 하나만 있는 경우 반복자 대신 배치를 직접 제공하므로 이 경우 더 합리적일 수 있습니다. 로드할 데이터 배치가 여러 개인 경우 로더를 다시 반환합니다.



인수(Args):@@@@
* attributes (list or dict, optional):
        Edge attributes to be included. If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load all edges at once directly to local. Default.

In [6]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=1,
    attributes=["time", "is_train"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 227 ms, sys: 48.6 ms, total: 276 ms
Wall time: 45.9 s


In [7]:
%%time
# Get the only batch of data via the `data` property
data = edge_loader.data
data.shape

CPU times: user 11.9 ms, sys: 3.22 ms, total: 15.1 ms
Wall time: 36.8 ms


(10556, 4)

In [8]:
data.head()

,source,target,time,is_train
0,10485760,14680110,0,0
1,10485761,1048637,0,1
2,10485761,32505881,0,1
3,10485762,12583005,0,0
4,10485763,14680101,0,1


#### Get batches of edges through http

In [9]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None
)

CPU times: user 4.46 ms, sys: 1.35 ms, total: 5.81 ms
Wall time: 8.78 ms


In [10]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (1010, 4)----
    source    target  time  is_train
0  1048576  23068756     0         1
----Batch 1: Shape (1119, 4)----
    source    target  time  is_train
0  4194309  31457366     0         1
----Batch 2: Shape (1072, 4)----
     source    target  time  is_train
0  10485763  14680101     0         1
----Batch 3: Shape (1008, 4)----
    source    target  time  is_train
0  4194310  20971534     0         0
----Batch 4: Shape (1115, 4)----
     source   target  time  is_train
0  11534338  3145787     0         0
----Batch 5: Shape (983, 4)----
    source    target  time  is_train
0  3145737  18874400     0         1
----Batch 6: Shape (1103, 4)----
    source    target  time  is_train
0  7340042  26214420     0         1
----Batch 7: Shape (1023, 4)----
    source   target  time  is_train
0  5242882  4194363     0         1
----Batch 8: Shape (1035, 4)----
    source    target  time  is_train
0  3145732  12582928     0         1
----Batch 9: Shape (1088, 4)----
     

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [11]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [12]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [13]:
%%time
loader = conn.gds.edgeLoader(
    attributes={"v0v0": ["is_train", "is_val"],
                "v2v0": ["is_train", "is_val"]},
    batch_size=200,
    shuffle=False,
    filter_by=None,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 27.8 ms, sys: 4.36 ms, total: 32.1 ms
Wall time: 32.7 s


In [14]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Edge type:", j)
        print(batch[j].head(1))

----Batch 0----
Edge type: v0v0
      source     target is_train is_val
0  137363456  136314881        0      0
Edge type: v2v0
      source     target is_train is_val
0  201326594  149946369        0      0
----Batch 1----
Edge type: v0v0
      source     target is_train is_val
0  135266305  134217729        0      0
Edge type: v2v0
      source     target is_train is_val
0  204472321  134217729        0      0
----Batch 2----
Edge type: v0v0
      source     target is_train is_val
0  135266304  147849218        0      0
Edge type: v2v0
      source     target is_train is_val
0  206569473  158334976        0      0
----Batch 3----
Edge type: v0v0
      source     target is_train is_val
0  134217728  136314884        0      0
Edge type: v2v0
      source     target is_train is_val
0  202375169  137363460        0      0
----Batch 4----
Edge type: v0v0
      source     target is_train is_val
0  157286401  154140672        0      0
Edge type: v2v0
      source     target is_train is_val


### Streaming through Kafka

**Note**: Kafka streaming function is only available for the Enterprise Edition. You need to activate the Enterprise Edition to use it. 

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official [doc](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) for detailed settings.

In [4]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### Get batches of edges

In [5]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 160 ms, sys: 34.8 ms, total: 195 ms
Wall time: 1min 4s


In [ ]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))